In [1]:
# -
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row, gridplot
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

import fire
import os
import h5py
import numpy as np

from pprint import pprint

from alphalogical.util import butter_bandpass_filter
from alphalogical.util import hilbert_power

from alphalogical.util import load_good_smith_files
from alphalogical.util import load_smith_foof_results
from alphalogical.util import load_smith_rates
from alphalogical.util import load_smith_lfps

from alphalogical.util import find_bursts
from alphalogical.util import find_extrema

from alphalogical.util import analyze_bursts
from alphalogical.util import shift_bursts
from alphalogical.util import shift_extrema
from alphalogical.util import analyze_extrema
from alphalogical.util import write_analysis

Loading BokehJS ...

ALPHA_CACHEDIR was not preset.
ALPHA_CACHEDIR set to /var/folders/s6/8n6rytmn3xq5q0wgkxktwbs00000gn/T/tmpg3LL6J


# Pathing

In [2]:
DATA_PATH = "/Users/type/Data/Smith/data"


# Load data and segment files

In [3]:
lfp_files, rate_files, seg_files = load_good_smith_files(DATA_PATH)

# Pick the dataset to analyze 

Name it `x`.

In [4]:
n = 0
lfp_file = lfp_files[n]
rate_file = rate_files[n]
seg_file = seg_files[n]

In [5]:
# Load all its segments
header, segments = load_smith_foof_results(seg_file)

print("Header {}".format(header))
print("Segment N: {}".format(len(segments)))
print("Segment row {}".format(segments[0]))

Header ['m', 'c', 'i', 'j', 'center', 'power', 'bw']
Segment N: 11805
Segment row (0, 49, 1131712, 1141712, 13.0381994977, 0.3680282355, 2.83154123279)


# Pick a segment

In [6]:
m = 2004

# Step through the major parts of `smith_burst_analysis.py`

In [7]:
# Get indices
m, c, i, j, center, power, bw = segments[m]

In [ ]:
# %%timeit load_smith_lfps(lfp_file, c)

In [22]:
fs, times, lfp = load_smith_lfps(lfp_file, c)
T = times.max()

In [ ]:
# %%timeit load_smith_rates(rate_file, c, (0, T), 1 / fs)

In [23]:
_, rate = load_smith_rates(rate_file, c, (0, T), 1 / fs)

# pad rate to match lfp (1 bin correction)
rate = np.pad(rate, (1, 0), 'constant')

In [25]:
# Do filtering over the whole electrode, for consistency
# -
# Alpha
alpha_range = (center - bw, center + bw)
alpha = butter_bandpass_filter(lfp, alpha_range[0], alpha_range[1], fs, order=2)

# Watch out for even signals....
if alpha.size % 2 == 0:
    alpha_pow = hilbert_power(alpha)
else:
    # Make even
    alpha_pow = hilbert_power(alpha[1:])
    # then pad
    alpha_pow = np.pad(alpha_pow, (0, 1), 'constant')

In [28]:
%timeit butter_bandpass_filter(lfp, alpha_range[0], alpha_range[1], fs, order=2)

10 loops, best of 3: 27.8 ms per loop


In [30]:
%timeit hilbert_power(alpha)

10 loops, best of 3: 53.3 ms per loop


In [31]:
# -
# HG
hg = butter_bandpass_filter(lfp, 100, 150, fs, order=2)

# Watch out for even signals....
if hg.size % 2 == 0:
    hg_pow = hilbert_power(hg)
else:
    # Make even
    hg_pow = hilbert_power(hg[1:])
    # then pad
    hg_pow = np.pad(hg_pow, (0, 1), 'constant')

hg_pow = hilbert_power(hg)

In [33]:
%timeit butter_bandpass_filter(lfp, 100, 150, fs, order=2)

10 loops, best of 3: 27.4 ms per loop


In [34]:
%timeit hg_pow = hilbert_power(hg)

10 loops, best of 3: 52.1 ms per loop


In [36]:
# --------------------------------------------------------------
# Find low
low_range = [center - (bw + 2), center - bw]
if low_range[0] < 0:
    low_range[0] = 1
if low_range[1] < 2:
    low_range[1] = 2
low = butter_bandpass_filter(
    lfp, low_range[0], low_range[1], fs, order=2)
low = hilbert_power(low)

# Find high
high_range = [center + bw, center + (bw + 2)]
high = butter_bandpass_filter(
    lfp, high_range[0], high_range[1], fs, order=2)
high = hilbert_power(high)

In [37]:
# --------------------------------------------------------------
# Only run for this mth segment...
# Bursts
M = np.mean((low + high) / 2.0)
bursts = find_bursts(alpha_pow[i:j], 3 * M, 1.5 * M)

In [38]:
%timeit find_bursts(alpha_pow[i:j], 3 * M, 1.5 * M)

100 loops, best of 3: 4.25 ms per loop


In [40]:
# Peaks
peaks, troughs = find_extrema(
    alpha[i:j],
    fs,
    alpha_range,
    boundary=None,
    first_extrema='peak')

In [42]:
%timeit find_extrema(alpha[i:j], fs, alpha_range, boundary=None, first_extrema='peak')

1000 loops, best of 3: 789 µs per loop


In [43]:
# Shift indices to match data
bursts = shift_bursts(bursts, i)
peaks = shift_extrema(peaks, i)
troughs = shift_extrema(troughs, i)